## Comparison Between Different Learning rates
---

### Learning Rates in Experiment
[0.01, 0.03, 0.07, 0.001, 0.0007, 0.0001]

### 1. Load modules and datasets

In [ ]:
# Set path of this notebook to the root directory
import os
os.chdir(os.path.join(os.getcwd(), '..'))
print 'Current working directory', os.getcwd()

In [ ]:
# For auto-reloading external modules
%load_ext autoreload
%autoreload 2

# Import modules
import pandas as pd
import numpy as np
import data_utils as du
import neural_network as nn
from sklearn.model_selection import KFold


# Prepare datasets for processing
du.maybe_pickle('train.csv')
du.maybe_pickle('test.csv')

train_dataset = du.load_dataset('train')
test_dataset = du.load_dataset('test')

print
print 'Train dataset has shape =', train_dataset[0].shape
print 'Test dataset has shape =', test_dataset[0].shape

### 2. Configure Network

In [ ]:
optimizer = 'RMSPropOptimizer'
num_steps = 5000
hidden_dim = (1024, 512, 64)
batch_size = 256
val_folds = 7    # Only fold 0 will be used (no cross-validation)

# Configure the following lines in affine_layer_variables() in layer_utils.py
# w_init = tf.truncated_normal([matrix_dim0, matrix_dim1], stddev=0.1)
# b_init = tf.zeros([matrix_dim1])

### 3. Prepare Testing Variables and Functions

In [ ]:
def run_network(learning_rate):
    
    kf = KFold(n_splits=val_folds)
    train_idx, val_idx = kf.split(train_dataset[0]).next()
    
    
    clf = nn.NeuralNetwork(hidden_dim=hidden_dim, optimizer=optimizer, 
                 learning_rate=learning_rate, num_steps=num_steps,
                 batch_size=batch_size)
    
    train = du.get_batch(train_dataset, indices=train_idx)
    val = du.get_batch(train_dataset, indices=val_idx)
    
    clf.fit(train, val, 'log/learning/rate' + str(learning_rate) + '/')
    train_acc = clf.accuracy(train)
    val_acc = clf.accuracy(val)

    print 'accuracy on train dataset = %.4f' % train_acc
    print 'accuracy on validation dataset = %.4f' % val_acc

### 4. Run Network for Every Hidden Layers Configuration

In [ ]:
learning_rates = [0.001, 0.003, 0.007, 0.01]

for learning_rate in learning_rates:
    print "Running learning rate %.3f" % learning_rate
    run_network(learning_rate)
    print 

### 5. Plotting Results

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def getData(path):
    """
        Returns data from CSV file exported from Tensorboard. The returned data is an np array.
    """
    data = pd.read_csv(path).iloc[:, 1:].as_matrix()
    return data

In [ ]:
# Plot loss for learning rates = [0.01, 0.001, 0.0001]
l_rates = [0.01, 0.001, 0.0001]
colors = ['#F5DB24', '#C91AE5', '#FF5733']
for idx, l_rate in enumerate(l_rates):
    loss_data = getData('plotting/data_csv/learning/loss/rate' + str(l_rate) + '.csv')
    x, y = loss_data[:, 0], loss_data[:, 1]
    plt.plot(x, y, color=colors[idx], label= 'rate %.4f' % l_rate)
plt.title('Loss Function for Learning Rates')
plt.legend()
plt.xlabel('steps')
plt.ylabel('loss')
plt.axis([0, 5000, 0, 1.5])
plt.grid(True)
plt.show()

In [ ]:
# Plot val accuracies for learning rates = [0.01, 0.001, 0.0001]
l_rates = [0.01, 0.001, 0.0001]
colors = ['#F5DB24', '#C91AE5', '#FF5733']
for idx, l_rate in enumerate(l_rates):
    val_acc = getData('plotting/data_csv/learning/val_acc/rate' + str(l_rate) + '.csv')
    x, y = val_acc[:, 0], val_acc[:, 1]
    plt.plot(x, y, color=colors[idx], label= 'rate %.4f' % l_rate)
plt.title('Validation Accuracy for Learning Rates')
plt.legend(bbox_to_anchor=(0., .12, 1., .102))
plt.xlabel('steps')
plt.ylabel('validation accuracy')
plt.axis([0, 5000, 0.5, 1])
plt.grid(True)
plt.show()